In [47]:
# !pip install openai aiohttp nest_asyncio

import asyncio
import json
import openai
import aiohttp
import nest_asyncio
from IPython.display import Markdown, display
import textwrap

nest_asyncio.apply()
openai.api_key = 'lm-studio'

code_highlight = ", the code in response will be like this ```{program_language_name}"

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

async def fetch_completion(my_msg, code_highlights = False):
    async with aiohttp.ClientSession() as session:
        async with session.post(
            'http://localhost:1234/v1/chat/completions',
            headers={
                # 'Authorization': f'Bearer {openai.api_key}',
                'Content-Type': 'application/json'
            },
            json={
                'model': 'meta-llama-3.1-8b-instruct',
                "messages": [
                  { "role": "system", "content": "if code is in response add the code like this ```{program_language_name}" },
                  { "role": "user", "content": my_msg if not code_highlights else my_msg + code_highlight }
                ],
                'temperature': 0.7,
                "max_tokens": -1, # -1 for unlimited
                "stream": True
            }
        ) as response:
            content = ""
            display_handle = display(Markdown(content), display_id=True)
            async for line in response.content:
                line = line.decode('utf-8').strip()
                if line.startswith("data: "):
                    data = line[len("data: "):]
                    if data != '[DONE]':
                        json_data = json.loads(data)
                        delta = json_data['choices'][0]['delta']
                        if 'content' in delta:
                            content += delta['content']
                            display_handle.update(to_markdown(content))
            return content

# prompt = "Once upon a time"
# full_ans = await fetch_completion(prompt)
# print(full_ans)


In [49]:
prompt = "write in max 100 words python code that get url data"
full_ans = await fetch_completion(prompt, code_highlights=True)
# print(full_ans)

> Here's a Python script using `requests` library to fetch URL data:
> 
> ```python
> import requests
> 
> def get_url_data(url):
>     try:
>         response = requests.get(url)
>         return response.text
>     except Exception as e:
>         print(f"Error: {e}")
>         return None
> 
> url = input("Enter the URL: ")
> data = get_url_data(url)
> 
> if data is not None:
>     print(data[:100])  # Print first 100 characters of the HTML content
> ```
> 
> To use this code, replace `requests` with `pip install requests` in your terminal. This script will fetch the HTML content from the entered URL and print it. Note that the output may be too large for some URLs; you can adjust the slicing as per your needs.

In [1]:
# first start the server and selecting module:
# cd C:\Users\yaniv\.cache\lm-studio\bin
# lms server start --port 1234
# select the module "LM Studio Community/Meta-Llama-3-8B-Instruct-GGUF":
# or by cmd: cd D:\Temp\ml_studio\TS\load_model; yarn start 
# pip install OpenAI
from openai import OpenAI
from IPython.display import Markdown
import textwrap

# install moduls path: C:\Users\yaniv\.cache\lm-studio\models\LM Studio Community\
# start LM Studio url: https://lmstudio.ai/
# select module
# start the server
# open url: http://localhost:1234/v1/models
# see server log in real time on respones
# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
code_highlight = ", the code in response will be like this ```{program_language_name}"

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def sendServer(my_msg, code_highlights = False):
  completion = client.chat.completions.create(
    model="LM Studio Community/Meta-Llama-3-8B-Instruct-GGUF",
    messages=[
        {"role": "system", "content": "if code is in response add the code like this ```{program_language_name}"},
        {"role": "user", "content": my_msg if not code_highlights else my_msg + code_highlight},
    ],
    temperature=0.7,
  )
  return to_markdown(completion.choices[0].message.content)


In [2]:
sendServer('''write in max 100 words python code that get url data''')

> **Get URL Data using Python**
> 
> You can use the `requests` library to get data from a URL.
> 
> ```python
> import requests
> 
> def get_url_data(url):
>     try:
>         response = requests.get(url)
>         if response.status_code == 200:
>             return response.json()
>         else:
>             print("Failed to retrieve data")
>             return None
>     except Exception as e:
>         print(f"An error occurred: {e}")
>         return None
> 
> # Example usage
> url = "https://jsonplaceholder.typicode.com/todos/1"
> data = get_url_data(url)
> print(data)
> ```
> 
> This code sends a GET request to the specified URL and returns the response data in JSON format.

In [47]:
sendServer('''write in max 100 words What is the meaning of life the answer need to be in points ?''')

> **The Meaning of Life:**
> 
> 1. **Purpose**: To fulfill one's potential and contribute to society.
> 2. **Happiness**: To find joy, contentment, and fulfillment in life.
> 3. **Relationships**: To build meaningful connections with others and experience love and belonging.
> 4. **Personal Growth**: To learn, adapt, and evolve as a person.
> 5. **Legacy**: To leave a positive impact on the world and be remembered after one's passing.
> 6. **Spirituality**: To connect with something greater than oneself, finding meaning and purpose in the universe.
> 7. **Self-actualization**: To realize one's full potential and become the best version of themselves.
> 
> These points highlight various aspects that contribute to a fulfilling life.

In [53]:
# how to force code color if defalt not working 
sendServer('''write in max 100 words python code how to convert json to dict''', True)

> Here is a Python code snippet that converts JSON to a dictionary:
> 
> ```python
> import json
> 
> json_string = '{"name": "John", "age": 30, "city": "New York"}'
> json_data = json.loads(json_string)
> 
> print(json_data)
> ```
> 
> This code uses the `json` module's `loads()` function to parse the JSON string into a Python dictionary. The resulting dictionary is then printed to the console.

In [15]:
# test
sendServer('''write in max 100 words python code that get url data''')

> Here is a Python code snippet using `requests` library to fetch URL data:
> ```
> import requests
> 
> url = "https://example.com"  # replace with your desired URL
> response = requests.get(url)
> 
> if response.status_code == 200:
>     print("URL Data:")
>     print(response.text)
> else:
>     print(f"Failed to retrieve data from {url} - Status Code: {response.status_code}")
> ```
> This code sends a GET request to the specified URL and prints the response text if the status code is 200 (OK). If not, it prints an error message with the status code.